In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, MultiHeadAttention, LayerNormalization, Dense, Dropout
import numpy as np

# Positional Encoding
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, embedding_dim):
        super(PositionalEncoding, self).__init__()
        self.sequence_length = sequence_length
        self.embedding_dim = embedding_dim
        self.positional_encoding = self.compute_positional_encoding()

    def compute_positional_encoding(self):
        pos = np.arange(self.sequence_length)[:, np.newaxis]
        i = np.arange(self.embedding_dim)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(self.embedding_dim))
        angle_rads = pos * angle_rates

        # Apply sin to even indices and cos to odd indices
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        positional_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(positional_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.positional_encoding[:, :tf.shape(inputs)[1], :]

# Transformer Encoder Layer
class TransformerEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embedding_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training, mask):
        # Multi-head self-attention
        attn_output = self.mha(inputs, inputs, inputs, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        # Feed-forward network
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Transformer Decoder Layer
class TransformerDecoderLayer(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        self.mha2 = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(embedding_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.layernorm3 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        self.dropout3 = Dropout(rate)

    def call(self, inputs, enc_output, training, look_ahead_mask, padding_mask):
        # Multi-head self-attention (for the target sequence)
        attn1 = self.mha1(inputs, inputs, inputs, attention_mask=look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(inputs + attn1)

        # Multi-head cross-attention (attends to the encoder output)
        attn2 = self.mha2(out1, enc_output, enc_output, attention_mask=padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(out1 + attn2)

        # Feed-forward network
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        return self.layernorm3(out2 + ffn_output)

# Transformer Model
def build_transformer(vocab_size, sequence_length, embedding_dim, num_heads, ff_dim, num_layers, rate=0.1):
    inputs = tf.keras.Input(shape=(sequence_length,))
    targets = tf.keras.Input(shape=(sequence_length,))

    # Embedding + Positional Encoding for input and target sequences
    enc_embedding = Embedding(vocab_size, embedding_dim)(inputs)
    dec_embedding = Embedding(vocab_size, embedding_dim)(targets)

    enc_positional = PositionalEncoding(sequence_length, embedding_dim)(enc_embedding)
    dec_positional = PositionalEncoding(sequence_length, embedding_dim)(dec_embedding)

    enc_output = enc_positional
    for _ in range(num_layers):
        enc_output = TransformerEncoderLayer(embedding_dim, num_heads, ff_dim, rate)(enc_output, training=True, mask=None)

    dec_output = dec_positional
    for _ in range(num_layers):
        dec_output = TransformerDecoderLayer(embedding_dim, num_heads, ff_dim, rate)(dec_output, enc_output, training=True, look_ahead_mask=None, padding_mask=None)

    outputs = Dense(vocab_size)(dec_output)

    model = tf.keras.Model(inputs=[inputs, targets], outputs=outputs)
    return model

# Example hyperparameters
vocab_size = 10000
sequence_length = 20
embedding_dim = 512
num_heads = 8
ff_dim = 1024
num_layers = 4

# Build the model
transformer = build_transformer(vocab_size, sequence_length, embedding_dim, num_heads, ff_dim, num_layers)

# Compile the model
transformer.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
transformer.summary()


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 20)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 20, 512)        │      5,120,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_encoding       │ (None, 20, 512)        │              0 │ embedding[0][0]        │
│ (PositionalEncoding)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_layer │ (None, 20, 512)        │      9,453,568 │ positional_encoding[0… │
│ (TransformerEncoderLayer) │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 20)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_laye… │ (None, 20, 512)        │      9,453,568 │ transformer_encoder_l… │
│ (TransformerEncoderLayer) │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 20, 512)        │      5,120,000 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_laye… │ (None, 20, 512)        │      9,453,568 │ transformer_encoder_l… │
│ (TransformerEncoderLayer) │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_encoding_1     │ (None, 20, 512)        │              0 │ embedding_1[0][0]      │
│ (PositionalEncoding)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_laye… │ (None, 20, 512)        │      9,453,568 │ transformer_encoder_l… │
│ (TransformerEncoderLayer) │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_decoder_layer │ (None, 20, 512)        │     17,856,000 │ positional_encoding_1… │
│ (TransformerDecoderLayer) │                        │                │ transformer_encoder_l… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_decoder_laye… │ (None, 20, 512)        │     17,856,000 │ transformer_decoder_l… │
│ (TransformerDecoderLayer) │                        │                │ transformer_encoder_l… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_decoder_laye… │ (None, 20, 512)        │     17,856,000 │ transformer_decoder_l… │
│ (TransformerDecoderLayer) │                        │                │ transformer_encoder_l… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_decoder_laye… │ (None, 20, 512)        │     17,856,000 │ transformer_decoder_l… │
│ (TransformerDecoderLayer) │                        │                │ transformer_encoder_l… │
├──────────────────────

 Total params: 124,608,272 (475.34 MB)

 Trainable params: 124,608,272 (475.34 MB)

 Non-trainable params: 0 (0.00 B)

In [1]:
# prompt: see gpu type

!nvidia-smi


Sun Oct  6 20:04:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [18]:
# Install the necessary libraries first if you haven't already
# !pip install transformers datasets torch scikit-learn

import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from datasets import load_dataset

# Load the IMDB dataset
dataset = load_dataset("imdb")

# Initialize TinyBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = AutoModelForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=2)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

# Tokenizing the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Creating DataLoaders
train_loader = DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True)
test_loader = DataLoader(tokenized_datasets["test"], batch_size=16)

# Moving model to GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Defining the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(3):  # Number of epochs
    for batch in train_loader:
        optimizer.zero_grad()  # Clear gradients
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")

# Evaluation
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()

        predictions.extend(predicted_labels)
        true_labels.extend(batch['label'].numpy())

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1: Loss = 0.461618036031723
Epoch 2: Loss = 0.013853178359568119
Epoch 3: Loss = 0.9304459691047668
Test Accuracy: 0.88
